# 코스피200 시가총액 크롤링

### 필요한 라이브러리

In [9]:
import requests
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from collections import defaultdict
import csv
import re
from scipy import stats

### 코스피 200 편입 종목들의 시가총액 크롤링

In [2]:
BaseUrl = 'http://finance.naver.com/sise/entryJongmok.nhn?&page='

# 종목 코드 추출
stock_codes = []

for i in range(1, 21):  # range() 함수의 끝 파라미터는 포함되지 않는다.
    url = BaseUrl + str(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    items = soup.find_all('td', {'class': 'ctg'})
    for item in items:
        txt = item.a.get('href')  # 예: https://finance.naver.com/item/main.nhn?code=006390
        k = re.search('[\d]+', txt)  # 정규표현식 사용. [\d] 숫자표현, + : 반복
        if k:
            code = k.group()
            stock_codes.append(code)

print(stock_codes)

<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_98635/155905241.py:13: SyntaxWarning: invalid escape sequence '\d'
  k = re.search('[\d]+', txt)  # 정규표현식 사용. [\d] 숫자표현, + : 반복


['005930', '000660', '207940', '373220', '005380', '012450', '329180', '105560', '000270', '068270', '035420', '055550', '042660', '012330', '011200', '034020', '028260', '009540', '138040', '005490', '086790', '259960', '015760', '000810', '010130', '032830', '035720', '033780', '051910', '402340', '316140', '096770', '267260', '030200', '010140', '064350', '024110', '066570', '352820', '006400', '017670', '323410', '003550', '018260', '034730', '009150', '272210', '079550', '003230', '042700', '047810', '003670', '000100', '086280', '047050', '003490', '180640', '006800', '010120', '090430', '326030', '021240', '010620', '005830', '000150', '009830', '267250', '010950', '032640', '298040', '071050', '005940', '161390', '000720', '051900', '271560', '029780', '006260', '016360', '251270', '241560', '034220', '278470', '377300', '000880', '028050', '128940', '011790', '035250', '001040', '039490', '078930', '011070', '036570', '175330', '097950', '022100', '036460', '138930', '004020',

In [3]:
# 네이버 금융 URL 기본
base_url = 'https://finance.naver.com/item/main.naver?code='

# 시가총액을 저장할 리스트와 종목명 리스트
market_caps = []
stock_names = []

# User-Agent 헤더 추가
headers = {'User-Agent': 'Mozilla/5.0'}

# 각 종목코드에 대한 시가총액 크롤링
for code in stock_codes:
    try:
        url = base_url + code
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 종목명 가져오기
        name_tag = soup.select_one('div.wrap_company h2 a')
        if name_tag:
            stock_name = name_tag.text.strip()
            stock_names.append(stock_name)
        else:
            stock_names.append(f"Unknown-{code}")
        
        # 시가총액 태그 찾기
        market_cap_tag = soup.find('em', {'id': '_market_sum'})
        if market_cap_tag:
            market_cap_text = market_cap_tag.get_text(strip=True)
            
            # '조' 단위와 숫자 분리 처리
            if '조' in market_cap_text:
                trillion_part = market_cap_text.split('조')[0]
                billion_part = market_cap_text.split('조')[1].strip()
                
                # 억 단위 처리 (공백이나 쉼표 제거)
                billion_part = billion_part.replace(',', '').strip()
                
                # 조 단위 계산 (1조 = 10000억)
                market_cap_value = float(trillion_part) * 10000
                
                # 억 단위가 있으면 더하기
                if billion_part:
                    market_cap_value += float(billion_part)
            else:
                # '조' 단위가 없는 경우 (억 단위만 있는 경우)
                market_cap_value = float(market_cap_text.replace(',', ''))
            
            market_caps.append(market_cap_value)
        else:
            market_caps.append(0)  # 시가총액을 찾지 못한 경우
            
        # 너무 빠른 요청으로 차단되지 않게 약간의 지연 추가
        time.sleep(0.5)
        
    except Exception as e:
        print(f"Error processing code {code}: {e}")
        market_caps.append(0)  # 오류 발생 시 0으로 기록
        if len(stock_names) < len(market_caps):
            stock_names.append(f"Error-{code}")

# 결과를 데이터프레임으로 만들기
result_df = pd.DataFrame({
    '종목코드': stock_codes[:len(market_caps)],  # 길이 맞추기
    '종목명': stock_names,
    '시가총액(억원)': market_caps
})

# 결과 확인
print("\n크롤링한 시가총액 데이터:")
print(result_df.head(10))  # 상위 10개 종목만 출력



크롤링한 시가총액 데이터:
     종목코드        종목명   시가총액(억원)
0  005930       삼성전자  3362354.0
1  000660     SK하이닉스  1488765.0
2  207940   삼성바이오로직스   713163.0
3  373220   LG에너지솔루션   679770.0
4  005380        현대차   405849.0
5  012450  한화에어로스페이스   397761.0
6  329180    HD현대중공업   372403.0
7  105560       KB금융   368343.0
8  000270         기아   365859.0
9  068270       셀트리온   343404.0


In [4]:
result_df

,종목코드,종목명,시가총액(억원)
0,005930,삼성전자,3362354.0
1,000660,SK하이닉스,1488765.0
2,207940,삼성바이오로직스,713163.0
3,373220,LG에너지솔루션,679770.0
4,005380,현대차,405849.0
...,...,...,...
195,271940,일진하이솔루스,5004.0
196,002710,TCC스틸,4850.0
197,093370,후성,4430.0
198,105630,한세실업,4252.0


### 시가 총액 기준으로 Z-score 3이상인 기업 찾기

In [10]:
# Z-score 계산
result_df['Z-score'] = stats.zscore(result_df['시가총액(억원)'])

# Z-score가 3 이상인 종목 필터링
outliers = result_df[result_df['Z-score'] >= 3]

# Z-score가 3 이상인 종목들의 리스트 생성
outlier_stocks = list(zip(outliers['종목코드'], outliers['종목명'], outliers['시가총액(억원)'], outliers['Z-score']))

print("\nZ-score가 3 이상인 종목들:")
for code, name, cap, zscore in outlier_stocks:
    print(f"종목코드: {code}, 종목명: {name}, 시가총액: {cap:.2f}억원, Z-score: {zscore:.2f}")

print(f"\n총 {len(outlier_stocks)}개 종목이 Z-score 3 이상의 시가총액을 가지고 있습니다.")


Z-score가 3 이상인 종목들:
종목코드: 005930, 종목명: 삼성전자, 시가총액: 3362354.00억원, Z-score: 12.01
종목코드: 000660, 종목명: SK하이닉스, 시가총액: 1488765.00억원, Z-score: 5.12

총 2개 종목이 Z-score 3 이상의 시가총액을 가지고 있습니다.
